In [1]:
!pip install pyvi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 44.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 993.5/993.5 kB 44.4 MB/s eta 0:00:00


In [2]:
import os
import re
import json
import string
import emoji
import pandas as pd
from vabsa.bartpho.preprocess import normalize, tokenize
from vabsa.bartpho.utils import tag_dict, polarity_dict, polarity_list, tags, eng_tags, eng_polarity, detect_labels, no_polarity, no_tag
from vabsa.bartpho.utils import predict, predict_df
from pyvi.ViTokenizer import tokenize as model_tokenize

In [8]:
data=open(r'datasets/orig/VLSP2018-SA-Restaurant-train.txt').read()

opinions = data.split('\n\n')
    
sentences, tags_lst, sentiments_lst, lengths = [], [], [], []
#count = 0
for i in opinions:
    #count += 1
    splitt = i.split("\n")
    num, sentence, aspects = splitt[0], splitt[1], splitt[2]
    sentence = normalize(sentence)
    sentence = tokenize(sentence, model_tokenize)
    aspects = split_aspect(aspects)
    text_tags = [x[0] for x in aspects]
    sentiments = [x[1] for x in aspects]
    sentences.append(sentence)
    tags_lst.append(text_tags)
    sentiments_lst.append(sentiments)
    lengths.append(len(sentence.split()))

no_sentences = len(sentences) 
print(no_sentences)
print(max(lengths))
print(sum(lengths)/no_sentences)

'''short_ids = [i for i in range(no_sentences) if lengths[i] <= 128]

for i in range(no_sentences):
    if i not in short_ids:
        continue
    else:
        short_ids.remove(i)
        for j in range(no_sentences):
            if j in short_ids:
                cmm = [x for x in tags_lst[i] if x in tags_lst[j]]
                if len(cmm) == 0:
                    sentences.append(sentences[i] + " " + sentences[j])
                    tags_lst.append(tags_lst[i] + tags_lst[j])
                    sentiments_lst.append(sentiments_lst[i] + sentiments_lst[j])
                    lengths.append(lengths[i] + lengths[j])
                    short_ids.remove(j)
                    continue'''
                    
sentence_lst, targets= [], []
for i in range(len(sentences)):
    for tag in tag_dict:
        sentence = sentences[i] 
        if tag not in tags_lst[i]:
            sentiment = polarity_dict['không có']
            target = "Nhận_xét " + tag_dict[tag] + " " + sentiment + " ."
            
        else:
            idx = tags_lst[i].index(tag)
            target = "Nhận_xét " + tag_dict[tag] + " " + polarity_dict[sentiments_lst[i][idx]]+ " ."
        sentence_lst.append(sentence)
        targets.append(target)
        
df = pd.DataFrame()
df['input_text'] = sentence_lst
df['target_text'] = targets
df.to_csv("bartpho_res_train.csv", index = False)

2961
615
58.211077338736914


In [ ]:
data=open(r'datasets/orig/VLSP2018-SA-Restaurant-train.txt').read()

opinions = data.split('\n\n')
    
sentences, tags_lst, sentiments_lst, lengths = [], [], [], []
#count = 0
for i in opinions:
    #count += 1
    splitt = i.split("\n")
    num, sentence, aspects = splitt[0], splitt[1], splitt[2]
    sentence = normalize(sentence)
    sentence = tokenize(sentence, model_tokenize)
    aspects = split_aspect(aspects)
    text_tags = [x[0] for x in aspects]
    sentiments = [x[1] for x in aspects]
    sentences.append(sentence)
    tags_lst.append(text_tags)
    sentiments_lst.append(sentiments)
    lengths.append(len(sentence.split()))

no_sentences = len(sentences) 
print(no_sentences)
print(max(lengths))
print(sum(lengths)/no_sentences)

'''short_ids = [i for i in range(no_sentences) if lengths[i] <= 128]

for i in range(no_sentences):
    if i not in short_ids:
        continue
    else:
        short_ids.remove(i)
        for j in range(no_sentences):
            if j in short_ids:
                cmm = [x for x in tags_lst[i] if x in tags_lst[j]]
                if len(cmm) == 0:
                    sentences.append(sentences[i] + " " + sentences[j])
                    tags_lst.append(tags_lst[i] + tags_lst[j])
                    sentiments_lst.append(sentiments_lst[i] + sentiments_lst[j])
                    lengths.append(lengths[i] + lengths[j])
                    short_ids.remove(j)
                    continue'''
                    
sentence_lst, targets= [], []
for i in range(len(sentences)):
    for tag in tag_dict:
        sentence = sentences[i] 
        if tag not in tags_lst[i]:
            detect_target = tag_dict[tag] + " không được nhận_xét ."
            #sentiment = polarity_dict['không có']
            #target = "Nhận_xét " + tag_dict[tag] + " " + sentiment + " ."
            
        else:
            idx = tags_lst[i].index(tag)
            absa_target = "Nhận_xét " + tag_dict[tag] + " " + polarity_dict[sentiments_lst[i][idx]]+ " ."
            sentence_lst.append(sentence)
            targets.append(absa_target)
            detect_target = tag_dict[tag] + " có được nhận_xét ."
        sentence_lst.append(sentence)
        targets.append(detect_target)
        
df = pd.DataFrame()
df['input_text'] = sentence_lst
df['target_text'] = targets
df.to_csv("bartpho_res_train_detect.csv", index = False)

In [ ]:
data=open(r'datasets/orig/VLSP2018-SA-Restaurant-train.txt').read()
opinions = data.split('\n\n')
sentences = []
aspect_cols = [[] for i in range(len(tags))]

for i in opinions:
    splitt = i.split("\n")
    num, sentence, aspects = splitt[0], splitt[1], splitt[2]
    sentence = normalize(sentence)
    sentence = tokenize(sentence, model_tokenize)
    aspects = split_aspect(aspects)
    text_tags = [x[0] for x in aspects]
    sentiments = [x[1] for x in aspects]
    sentences.append(sentence)
    for j in range(len(tags)):
        tag = eng_tags[j]
        if tag not in text_tags:
            aspect_cols[j].append(0) 
        else:
            idx = text_tags.index(tag)
            aspect_cols[j].append(eng_polarity.index(sentiments[idx]))

df = pd.DataFrame()
df['text'] = sentences
for i in range(len(tags)):
    df[eng_tags[i]] = aspect_cols[i]
df.to_csv("res_train.csv", index = False)
with open('sentences.json', 'w') as f:
    json.dump(sentences, f, ensure_ascii=False, indent=4)

In [ ]:
df